In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('once')
from sklearn.tree import DecisionTreeRegressor
import datetime as dt
import pickle

In [11]:
df = pd.read_csv('../datos/cnt/11-cnt-cod_est.csv', index_col=0)
df.head(2)

,atemp,hum,windspeed,season,yr,mnth,holiday,weekday,weathersit,cnt
0,-0.456399,0.855582,-0.208909,0,0,0,0,0,1,985
1,-0.492863,0.332722,0.687550,0,0,0,2,0,1,801


In [12]:
df.dtypes

atemp         float64
hum           float64
windspeed     float64
season          int64
yr              int64
mnth            int64
holiday         int64
weekday         int64
weathersit      int64
cnt             int64
dtype: object

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   atemp       730 non-null    float64
 1   hum         730 non-null    float64
 2   windspeed   730 non-null    float64
 3   season      730 non-null    int64  
 4   yr          730 non-null    int64  
 5   mnth        730 non-null    int64  
 6   holiday     730 non-null    int64  
 7   weekday     730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   cnt         730 non-null    int64  
dtypes: float64(3), int64(7)
memory usage: 62.7 KB


In [14]:
df.head(2)

,atemp,hum,windspeed,season,yr,mnth,holiday,weekday,weathersit,cnt
0,-0.456399,0.855582,-0.208909,0,0,0,0,0,1,985
1,-0.492863,0.332722,0.687550,0,0,0,2,0,1,801


---

In [15]:
X = df.drop("cnt", axis = 1)
y = df["cnt"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) 

In [17]:
arbol = DecisionTreeRegressor(random_state =0)
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [18]:
#fig = plt.figure(figsize = (10,6))
#tree.plot_tree(arbol, feature_names = x_train.columns, filled = True)
#plt.show()

In [19]:
max_features = np.sqrt(len(x_train.columns))
max_features

3.0

In [20]:
print(arbol.tree_.max_depth)

21


In [21]:
param = {"max_depth": [4,6,8,10,12],
        "max_features": [1,2,3,4],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [22]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")
gs.fit(x_train, y_train)



GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [4, 6, 8, 10, 12],
                         'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [23]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=12, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [24]:
#fig = plt.figure(figsize=(40, 20))
#tree.plot_tree(mejor_modelo, feature_names=x_train.columns, filled=True);

In [25]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [26]:

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [27]:
dt_results1 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "1_Dtree_CNT_est-cod")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,581.129404,678586.725826,823.763756,0.801748,test,1_Dtree_CNT_est-cod
1,553.991963,590977.252178,768.750449,0.845119,train,1_Dtree_CNT_est-cod


Random forest

In [28]:
X = df.drop("cnt", axis = 1)
y = df["cnt"]

In [29]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [30]:
param = {"max_depth": [4,6,8,10,12],
        "max_features": [1,2,3,4],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [31]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 


In [32]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [4, 6, 8, 10, 12],
                         'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [33]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=10, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [34]:
'''for arbol in tqdm(bosque.estimators_):
    fig = plt.figure(figsize=(40, 20))
    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)
    plt.show()'''

'for arbol in tqdm(bosque.estimators_):\n    fig = plt.figure(figsize=(40, 20))\n    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)\n    plt.show()'

In [35]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
   
   
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df


In [36]:
dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "2_RForest_CNT_cod_estand")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,484.150958,395785.995639,629.115248,0.884369,test,2_RForest_CNT_cod_estand
1,499.781679,477244.148465,690.828596,0.874926,train,2_RForest_CNT_cod_estand


In [37]:
df_todos_resultados = pd.concat([dt_results, dt_results1], axis = 0)
df_todos_resultados


,MAE,MSE,RMSE,R2,set,modelo
0,484.150958,395785.995639,629.115248,0.884369,test,2_RForest_CNT_cod_estand
1,499.781679,477244.148465,690.828596,0.874926,train,2_RForest_CNT_cod_estand
0,581.129404,678586.725826,823.763756,0.801748,test,1_Dtree_CNT_est-cod
1,553.991963,590977.252178,768.750449,0.845119,train,1_Dtree_CNT_est-cod


In [38]:
df_todos_resultados.to_csv('../datos/cnt/12-cnt-metricas-estand.csv')

# Random Forest es la mejor opcion

In [42]:
with open('../datos/cnt/12-cnt-decision.pkl','wb') as fp:
    pickle.dump(arbol,fp)

In [43]:
with open('../datos/cnt/12-cnt-random.pkl','wb') as fp:
    pickle.dump(bosque,fp)